Following the tutorial at https://thedatafrog.com/en/articles/cuda-kernel-python/

In [2]:
%pylab inline
from numba import cuda, vectorize
import numpy as np
import math
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.0/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"

In [3]:
a = np.arange(4096*(2**5), dtype=np.float32)
a

array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 1.31069e+05,
       1.31070e+05, 1.31071e+05], dtype=float32)

In [4]:
%time _ = np.sqrt(a)

Wall time: 998 µs


In [5]:
@vectorize(['float32(float32)'], target='cuda')
def gpu_sqrt(x):
    return math.sqrt(x)

In [6]:
%time _ = gpu_sqrt(a)

Wall time: 1.02 s


In [7]:
@cuda.jit
def gpu_sqrt_kernel(x, out):
    idx = cuda.grid(1)
    out[idx] = math.sqrt(x[idx])

In [1]:
# move input data to the device
d_a = cuda.to_device(a)
d_out = cuda.device_array_like(d_a)

threads_per_block = 32
blocks_per_grid = math.ceil((len(a) + threads_per_block - 1)/threads_per_block)
gpu_sqrt_kernel[blocks_per_grid, threads_per_block](d_a, d_out)
print(blocks_per_grid)
#wait for all the threads to compute
#cuda.synchronize()
new_a = d_out.copy_to_host()
print(new_a)

NameError: name 'cuda' is not defined

In [9]:
(new_a-np.sqrt(a)).max()

0.0

In [10]:
cuda.detect()

Found 1 CUDA devices
id 0     b'GeForce RTX 2060'                              [SUPPORTED]
                      compute capability: 7.5
                           pci device id: 0
                              pci bus id: 1
Summary:
	1/1 devices are supported


True

This simple kernel deals with a single element of the input array:

In [11]:
@cuda.jit
def gpu_atan(x, out):
    idx = cuda.grid(1)
    out[idx] = math.atan(x[idx])

When the kernel is deployed, the GPU therefore needs to create as many threads as elements in the array, which potentially results in many blocks if the array is large.

On the contrary, a striding kernel deals with several elements of the input array, using a loop:

In [12]:
@cuda.jit
def gpu_atan_stride(x, out):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(start, x.shape[0], stride):
        out[i] = math.atan(x[i])

In this way, a given thread deals with several elements, and the number of threads is kept under control. Threads keep doing work in a coordinated way, and the GPU is not wasting time creating and scheduling threads.

Let's consider a small example with:

an input data array of size 8
blocks with 4 threads each
Without striding, we need to use two blocks so 8 threads in total, each dealing with a single element in the array:

In [13]:
a = np.arange(8, dtype=np.float32)
d_a = cuda.to_device(a)
d_out = cuda.device_array_like(d_a)

# 2 blocks of 4 threads
%time _ = gpu_atan[2,4](d_a, d_out)
new_a = d_out.copy_to_host()
print(d_out.copy_to_host())

Wall time: 79.8 ms
[0.        0.7853982 1.1071488 1.2490458 1.3258177 1.3734008 1.4056478
 1.4288993]


With striding, we can use a single block with 4 threads, in which case each thread deals with two elements. Specifically, in the code above:

start is the thread index, which is 0, 1, 2, 3 for the four threads, respectively.
stride is the size of the grid, which is the total number of threads in the grid, here 4.
x.shape[0] is the size of the input data array x , which is 8.

In [14]:
# 1 block of 4 threads
%time _ = gpu_atan_stride[1,4](d_a, d_out)
print(d_out.copy_to_host())

Wall time: 93.8 ms
[0.        0.7853982 1.1071488 1.2490458 1.3258177 1.3734008 1.4056478
 1.4288993]


# Performance analysis ¶
In this section, we will study the influence of striding and of the execution configuration parameters in the processing of a large array.

So let's redefine our kernels (the simple one and the striding one:)

In [15]:
@cuda.jit
def gpu_atan(x, out):
    idx = cuda.grid(1)
    out[idx] = math.atan(x[idx])

@cuda.jit
def gpu_atan_stride(x, out):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(start, x.shape[0], stride):
        out[i] = math.atan(x[i])

Now we create a big array, we ship it to the device, and we create the output array on the device as usual:

In [16]:
a = np.arange(256*1000000, dtype=np.float32)
d_a = cuda.to_device(a)
d_out = cuda.device_array_like(d_a)

First, let's see how fast we can process this array sequentially (in a single thread) on the GPU. We use the striding version here since, obviously, the simple version would only be able to process one element in a single thread.

In [17]:
%time gpu_atan_stride[1, 1](d_a, d_out); cuda.synchronize()

Wall time: 43.5 s


Processing these 256 million values in a single thread on the GPU took almost a minute. Let's see how parallel processing can help us. For that, we choose an execution configuration by following our simple rules, and we use the non-striding kernel:

In [18]:
%time gpu_atan[1000000,256](d_a, d_out); cuda.synchronize()

Wall time: 99.8 ms


In [19]:
%time np.arctan(a)

Wall time: 1.65 s


array([0.       , 0.7853982, 1.1071488, ..., 1.5707964, 1.5707964,
       1.5707964], dtype=float32)

Now, let's try and see if the striding version brings any performance improvement:

In [20]:
%time gpu_atan_stride[50000,256](d_a, d_out); cuda.synchronize()

Wall time: 12 ms


The gain is not very significant, you might have to rerun the two previous cells several times to convince yourself that there is indeed a gain. The amount of performance gain you'll get from striding will depend on the size of the input array, and on the work to be done by the kernel.

# Multidimensional datasets : Matrix multiplication ¶
So far, we've been working with one-dimensional arrays, making use of a 1D grid of threads.

In the following 1D kernel, cuda.grid(1) returns a single index that identifies the position of the thread in the grid, and and cuda.gridsize(1) returns the length of the grid of threads:

In [21]:
@cuda.jit
def gpu_atan_stride(x, out):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)
  for i in range(start, x.shape[0], stride): 
    out[i] = math.atan(x[i])

As we will see, these functions also provide an easy interface for the processing of 2D and 3D data.

In this section, you will learn how to deploy a 2D grid of threads to process 2D data, and how to stride through 2D data. Then, you will implement a practical case: matrix multiplication.

# A simple 2D kernel ¶
To learn the basic tools to work with 2D data, please consider the following kernel:

In [22]:
@cuda.jit
def gpu_2d(out):
    i1, i2 = cuda.grid(2)
    out[i1][i2] = i1*10 + i2

As you can see, this kernel does not even have an input. Its goal is only to encode and record in the output 2D array the coordinates of the current thread.

Now, let's create the output data structure:

In [23]:
a = np.zeros(12).reshape(3,4)
a

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

We copy this data structure to the device, and we deploy the kernel, before fetching the results:

In [24]:
d_a = cuda.to_device(a)

blocks = (1, 2)

threads_per_block = (3, 2)

gpu_2d[blocks, threads_per_block](d_a)

new_a = d_a.copy_to_host()
new_a

array([[ 0.,  1.,  2.,  3.],
       [10., 11., 12., 13.],
       [20., 21., 22., 23.]])

The first two columns were processed by the first block of threads, and the last two by the second block.

Looking at the kernel code and at the output above, we see that i1 indexes the first dimension (the lines) while i2 indexes the second dimension (the columns).

As in the 1D case, it's necessary to make sure that the grid covers the whole output data structure. For example, if we mess up in the description of the block structure, the last two columns remain unprocessed, and we actually access unallowed memory "below" our 2D array:

In [25]:
# need to send back the matrix of zeros 
# to reset it on the device
d_a = cuda.to_device(a)
# here we mess up: 
# we require two blocks on top of each other, 
# and we get a grid with a total height of 2x3=6 
# and a total width of 1x2=2...
gpu_2d[(2,1), (3,2)](d_a)
d_a.copy_to_host()

array([[ 0.,  1.,  0.,  0.],
       [10., 11.,  0.,  0.],
       [20., 21.,  0.,  0.]])

# 2D striding ¶
Striding in 2D is not much more difficult that in 1D. Again, let's take a very simple example, and first make sure that our implementation of striding is working:

In [26]:
@cuda.jit
def gpu_2d_stride(out):
    # Get the thread coordinates in 2d
    s1, s2 = cuda.grid(2)
    # get the grid sie in 2d
    d1, d2 = cuda.gridsize(2)
    for i1 in range(s1, out.shape[0], d1):
        for i2 in range(s2, out.shape[1], d2):
            out[i1][i2] = i1*10 + i2

We reset the output data on the device, and we deploy a single (3,2) block. The grid therefore covers only half of the output array. But the striding will move the grid in the output array, making sure that the whole array is covered:

In [27]:
d_a = cuda.to_device(a)
gpu_2d_stride[(1,), (3,2)](d_a)
new_a = d_a.copy_to_host()
new_a

array([[ 0.,  1.,  2.,  3.],
       [10., 11., 12., 13.],
       [20., 21., 22., 23.]])

We get the same results as without striding. Now let's visualize striding with a slightly different kernel, which will register the starting point (s1, s2) of each thread:

In [28]:
@cuda.jit
def gpu_2d_start(out):
    # get the thread coodinates in 2d
    s1, s2 = cuda.grid(2)
    # get the grid size in 2d
    d1, d2 = cuda.gridsize(2)
    for i1 in range(s1, out.shape[0], d1):
        for i2 in range(s2, out.shape[1], d2):
            #note the difference in using s rather than i
            out[i1][i2] = s1*10 + s2

In [29]:
d_a = cuda.to_device(a)
gpu_2d_start[(1,), (3,2)](d_a)
d_a.copy_to_host()

array([[ 0.,  1.,  0.,  1.],
       [10., 11., 10., 11.],
       [20., 21., 20., 21.]])

The elements with the same value are processed by the same thread, and we see that there is only one stride to the right.

Extending our 2D array, we see that striding occurs in both direction:

In [30]:
a = np.zeros(16).reshape(4,4)
d_a = cuda.to_device(a)
gpu_2d_start[(1,), (3,2)](d_a)
d_a.copy_to_host()

array([[ 0.,  1.,  0.,  1.],
       [10., 11., 10., 11.],
       [20., 21., 20., 21.],
       [ 0.,  1.,  0.,  1.]])

# 2D matrix multiplication : theory ¶
Implement a 2D matrix multiplication kernel is an excellent way to confirm that we now master striding in 2D.

First, a little reminder. Two matrices A of size (m,n) and B of size (n,p) can be multiplied since the number of colums of matrix A is equal to the number of lines of matrix B. The product matrix C is then of size (m,p). And the value of one of the coefficients of matrix C is given by:

 
This might seem a bit complicated if you are new to linear algebra or haven't used it in a long time. So let's take a simple example in code. We create two matrices:

In [31]:
a = np.arange(6).reshape(2,3)
b = np.arange(12).reshape(3,4)
print(a)
print(b)

[[0 1 2]
 [3 4 5]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [32]:
c = a.dot(b)
c

array([[20, 23, 26, 29],
       [56, 68, 80, 92]])

Matrix a is of size (2,3) and matrix b of size (3,4) , so the product matrix is of size (2,4) .

Let's consider for example the coefficient on the first line and second column of the product matrix, which is 23. This is c01. The summation equation above tells us that this coefficient is obtained by taking the coefficients of the first line of matrix a, and by multiplying them with the coefficients of the second column of matrix b, respectively, before summing everything.

We get: c01 = a00*b01 + a01*b11 + a02*b21 = 0*1 + 1*5 + 2*9 = 23

# A 2D matrix multiplication kernel ¶
As a first step, we will implement matrix multiplication without striding. So we will need one thread for each element in the output matrix. Here is the kernel:

In [59]:
@cuda.jit
def multiply(a, b, c):
    i1, i2 = cuda.grid(2)
    coeff = 0
    for k in range(b.shape[0]):
        coeff += a[i1][k]*b[k][i2]
    c[i1, i2] = coeff

In [60]:
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
c = np.zeros((a.shape[0], b.shape[1]))
print(c.shape)
d_c = cuda.to_device(c)
threads_per_block = 4
blocks_per_grid_1 = math.ceil((c.shape[0] + threads_per_block - 1)/threads_per_block)
blocks_per_grid_2 = math.ceil((c.shape[1] + threads_per_block - 1)/threads_per_block)
print(d_c.copy_to_host())

(1024, 1024)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Lets try with a bigger array and see the time improvement

In [5]:
k = 8
a = np.arange(2**k).reshape(2**int(k/2), 2**int(k/2))
b = np.arange(2**k).reshape(2**int(k/2), 2**int(k/2))

In [62]:
%time a.dot(b)

Wall time: 0 ns


array([[  333312,   333808,   334304, ...,   347696,   348192,   348688],
       [  841216,   842736,   844256, ...,   885296,   886816,   888336],
       [ 1349120,  1351664,  1354208, ...,  1422896,  1425440,  1427984],
       ...,
       [15062528, 15092720, 15122912, ..., 15938096, 15968288, 15998480],
       [15570432, 15601648, 15632864, ..., 16475696, 16506912, 16538128],
       [16078336, 16110576, 16142816, ..., 17013296, 17045536, 17077776]])

In [63]:
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
c = np.zeros((a.shape[0], b.shape[1]))
print(c.shape)
d_c = cuda.to_device(c)

threads_per_block = 512
blocks_per_grid_1 = 2
blocks_per_grid_2 = 2

%time _ = multiply[(1, blocks_per_grid_2), (threads_per_block,threads_per_block)](d_a, d_b, d_c)
a2 = d_c.copy_to_host()
(a2-a.dot(b)).max()

(32, 32)


CudaAPIError: [1] Call to cuLaunchKernel results in CUDA_ERROR_INVALID_VALUE

-333312.0

In [54]:
c.shape

(1024, 1024)

In [55]:
512*2

1024

In [108]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [105]:
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
c = np.zeros((a.shape[0], b.shape[1]))
print(c.shape)
d_c = cuda.to_device(c)

threads_per_block = 32
blocks_per_grid_1 = 2000
blocks_per_grid_2 = 2000

%time _ = matmul[(blocks_per_grid_1, blocks_per_grid_2), (threads_per_block,threads_per_block)](d_a, d_b, d_c)
c = d_c.copy_to_host()
(c-a.dot(b)).max()

(128, 128)
Wall time: 0 ns


17179869184.0

In [103]:
a.dot(b)

array([[  88432640,   88440768,   88448896, ...,   89448640,   89456768,
          89464896],
       [ 221601792,  221626304,  221650816, ...,  224665792,  224690304,
         224714816],
       [ 354770944,  354811840,  354852736, ...,  359882944,  359923840,
         359964736],
       ...,
       [-445292544, -443236416, -441180288, ..., -188276544, -186220416,
        -184164288],
       [-312123392, -310050880, -307978368, ...,  -53059392,  -50986880,
         -48914368],
       [-178954240, -176865344, -174776448, ...,   82157760,   84246656,
          86335552]])

In [104]:
a2

array([[8.84326400e+07, 8.84407680e+07, 8.84488960e+07, ...,
        8.94486400e+07, 8.94567680e+07, 8.94648960e+07],
       [2.21601792e+08, 2.21626304e+08, 2.21650816e+08, ...,
        2.24665792e+08, 2.24690304e+08, 2.24714816e+08],
       [3.54770944e+08, 3.54811840e+08, 3.54852736e+08, ...,
        3.59882944e+08, 3.59923840e+08, 3.59964736e+08],
       ...,
       [1.67345766e+10, 1.67366328e+10, 1.67386889e+10, ...,
        1.69915926e+10, 1.69936488e+10, 1.69957049e+10],
       [1.68677458e+10, 1.68698183e+10, 1.68718908e+10, ...,
        1.71268098e+10, 1.71288823e+10, 1.71309548e+10],
       [1.70009149e+10, 1.70030038e+10, 1.70050927e+10, ...,
        1.72620269e+10, 1.72641158e+10, 1.72662047e+10]])

In [77]:
32*32

1024

In [3]:
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [6]:
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
c = np.zeros((a.shape[0], b.shape[1]))
print(c.shape)
d_c = cuda.to_device(c)

threads_per_block = 32
blocks_per_grid_1 = 20
blocks_per_grid_2 = 20

%time _ = fast_matmul[(blocks_per_grid_1, blocks_per_grid_2), (threads_per_block,threads_per_block)](d_a, d_b, d_c)
c = d_c.copy_to_host()
(c-a.dot(b)).max()

(16, 16)
Wall time: 298 ms


CudaAPIError: [700] Call to cuMemcpyDtoH results in UNKNOWN_CUDA_ERROR